In [1]:
# Run command below with docker installed to download and run a spark docker instance:
# docker run --name sparkbook -p 8881:8888 -v "$PWD":/home/jovyan/work jupyter/pyspark-notebook start.sh jupyter lab --LabApp.token=''
# Next, open a web browser and navigate to localhost:8881 to enter docker container. This notebook can then be run.

In [2]:
import sys
!{sys.executable} -m pip install nltk
#Required when running from a newly created spark docker container

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#required for nltk functionality

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.types import ArrayType, FloatType, StringType
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import sys
import string
from collections import defaultdict

In [5]:
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()

In [6]:
sc = spark.sparkContext
spk = SparkSession.builder.master('local').getOrCreate()

In [7]:
df = spark.read.csv("data/gaming_reviews.tsv", sep="\t", header=True, inferSchema=True)

In [8]:
df_ratings = df.select('star_rating', 'review_body').limit(1000)

In [9]:
stop_words = set(stopwords.words('english'))

In [89]:
def process_str(row):
    """Input: String
    Output: List of strings without punctuation
    
    Removes punctuation using functions from the strings library. Performs raw string operations in C via a lookup table for maximum performance."""
    word_list = row.translate(str.maketrans('', '', string.punctuation)).split(' ')
    return [x.lower() for x in word_list if x not in stop_words]

In [90]:
process = udf(process_str, ArrayType(StringType()))

In [12]:
df_new = df_ratings.withColumn('body_list', process(df_ratings['review_body']))
df_new = df_new.select('star_rating', 'body_list')

In [13]:
type(df_new)

pyspark.sql.dataframe.DataFrame

In [14]:
df_new.head()

Row(star_rating=5, body_list=['Used', 'Elite', 'Dangerous', 'mac', 'amazing', 'joystick', 'I', 'especially', 'love', 'twist', 'stick', 'different', 'movement', 'bindings', 'well', 'move', 'normal', 'way'])

In [15]:
word_dict = defaultdict(lambda: 0)

In [16]:
df_new.select('body_list').head()[0]

['Used',
 'Elite',
 'Dangerous',
 'mac',
 'amazing',
 'joystick',
 'I',
 'especially',
 'love',
 'twist',
 'stick',
 'different',
 'movement',
 'bindings',
 'well',
 'move',
 'normal',
 'way']

In [18]:
model = cv.fit(df_new)

NameError: name 'cv' is not defined

In [ ]:
result = model.transform(df_new)

In [ ]:
type(result)

In [96]:
df_big = df.withColumn('body_list', process(df['review_body'])).select('star_rating', 'body_list').limit(1399)
df_big.collect()[1397]

Row(star_rating=1, body_list=['the', 'part', 'holds', 'batteries', 'broke', 'within', 'two', 'days', 'the', 'batteries', 'messed', 'controller', 'do', 'not', 'buy'])

In [95]:
cv2 = CountVectorizer(inputCol="body_list", outputCol="features")
model2 = cv2.fit(df_big)

Py4JJavaError: An error occurred while calling o1818.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 180.0 failed 1 times, most recent failure: Lost task 1.0 in stage 180.0 (TID 490, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 334, in _batched
    for item in iterator:
  Fil

In [ ]:
result2 = model2.transform(df_new)

In [42]:
result2.head()

Row(star_rating=5, body_list=['Used', 'Elite', 'Dangerous', 'mac', 'amazing', 'joystick', 'I', 'especially', 'love', 'twist', 'stick', 'different', 'movement', 'bindings', 'well', 'move', 'normal', 'way'], features=SparseVector(5068, {0: 1.0, 20: 1.0, 22: 1.0, 27: 1.0, 50: 1.0, 154: 1.0, 305: 1.0, 367: 1.0, 439: 1.0, 698: 1.0, 776: 1.0, 959: 1.0, 1006: 1.0, 1445: 1.0, 1522: 1.0, 2375: 1.0, 4558: 1.0, 4831: 1.0}))